In [1]:
#time in traffic value is in seconds text is in minutes

In [71]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from census import Census
import gmaps
import requests
import json
from pprint import pprint
from config import (api_key, gkey)
c = Census(api_key, year=2016)
from datetime import datetime
import gmaps
from us import states
import time

In [3]:
file_to_load = pd.read_csv("bayarea_zipcodes.csv", dtype="object", encoding="utf-8")
zip_code_df = pd.DataFrame(file_to_load)
zip_code_df.head()
bay_area_zip = []
for index, row in zip_code_df.iterrows():
    bay_area_zip.append(row['ZIP'])
zip_code_df.head()

,PO_NAME,the_geom,ZIP,STATE,Area__,Length__
0,NAPA,MULTIPOLYGON (((-122.10329200180091 38.5132829...,94558,CA,12313263537,995176.225313
1,FAIRFIELD,MULTIPOLYGON (((-121.947475002335 38.301511000...,94533,CA,991786103.42,200772.556587
2,DIXON,MULTIPOLYGON (((-121.65335500334429 38.3133870...,95620,CA,7236949520.92,441860.2014
3,SONOMA,MULTIPOLYGON (((-122.406843003057 38.155681999...,95476,CA,3001414164.85,311318.546326
4,NAPA,MULTIPOLYGON (((-122.29368500225117 38.1552379...,94559,CA,1194301744.88,359104.646602


In [4]:
time_list = []
time_am= int(datetime(2018,10,8,7).timestamp())
time_pm = int(datetime(2018,10,8,16).timestamp())
for x in range(1,7) :
    time= datetime(2018,10,8,7, 0).timestamp()
    time_list.append(time_am+1800)
    time_list.append(time_pm+1800)
    #print(time)
time_list.append(time)
time_list.append(time_pm)
print(time_list)

#datetime.datetime(time_list[0]).timestamp()

[1539009000, 1539041400, 1539009000, 1539041400, 1539009000, 1539041400, 1539009000, 1539041400, 1539009000, 1539041400, 1539009000, 1539041400, 1539007200.0, 1539039600]


In [26]:
#testing variable so im not running long code
#start_loc = ["Alameda,CA","Solano","Sonoma","Napa County"]
start_loc = ["Alameda,CA","Contra Costa,CA","Marin County","Napa County","San Francisco","San Mateo","Santa Clara","Solano","Sonoma"]
end_loc = (list(reversed(start_loc)))
#end_loc = (list(reversed(bay_area_zip)))
time_list = [1539009000]
origin_Loc = []
destination_loc = []
trip_distance = []
trip_duration = []
traffic_duration = []
origin_lat = []
origin_long = []
dest_lat= []
dest_long= []
               
#target_url = (f'https://maps.googleapis.com/maps/api/distancematrix/json?origins=San Francisco,CA|Oakland,CA&destinations=San Jose,CA|Palo Alto,CA&departure_time=now&key={gkey}')

In [27]:
for time in time_list:

   # for s_loc in bay_area_zip:
    for s_loc in start_loc :
        for e_loc in end_loc:
            try:
                target_url = (f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={s_loc}&destinations={e_loc}&mode=driving,CA&departure_time={time}&traffic_model=pessimistic&key={gkey}')
                
                geo_data = requests.get(target_url).json()
                
                destination_loc.append(geo_data['destination_addresses'][0])
                origin_Loc.append(geo_data['origin_addresses'][0])
                trip_distance.append(geo_data['rows'][0]['elements'][0]['distance']['value'])
                traffic_duration.append(geo_data['rows'][0]['elements'][0]['duration_in_traffic']['value'])
                trip_duration.append(geo_data['rows'][0]['elements'][0]['duration']['value'])
                
                #two calls with the start and end point geo location
                target_url_origin = (f' https://maps.googleapis.com/maps/api/geocode/json?address={s_loc}&key={gkey}')
                target_url_exit = (f' https://maps.googleapis.com/maps/api/geocode/json?address={e_loc}&key={gkey}')
                geoloc_data_origin = requests.get(target_url_origin).json()
                geoloc_data_exit = requests.get(target_url_exit).json()
                origin_lat.append(geoloc_data_origin['results'][0]['geometry']['location']['lat'])
                origin_long.append(geoloc_data_origin['results'][0]['geometry']['location']['lng'])
                dest_lat.append(geoloc_data_exit['results'][0]['geometry']['location']['lat'])
                dest_long.append(geoloc_data_exit['results'][0]['geometry']['location']['lng'])
                pprint(geo_data)
               
            except(KeyError):
                print(f"something broke with either {s_loc} or {e_loc}")

{'destination_addresses': ['Sonoma, CA 95476, USA'],
 'origin_addresses': ['Alameda, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '84.9 km', 'value': 84929},
                         'duration': {'text': '1 hour 3 mins', 'value': 3786},
                         'duration_in_traffic': {'text': '1 hour 34 mins',
                                                 'value': 5611},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Solano County, CA, USA'],
 'origin_addresses': ['Alameda, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '92.0 km', 'value': 91968},
                         'duration': {'text': '1 hour 6 mins', 'value': 3962},
                         'duration_in_traffic': {'text': '1 hour 19 mins',
                                                 'value': 4746},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Santa Clara, CA, USA'],
 'origin_addresses': ['Alameda, CA, USA'],
 '

{'destination_addresses': ['Solano County, CA, USA'],
 'origin_addresses': ['Marin County, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '99.0 km', 'value': 98957},
                         'duration': {'text': '1 hour 19 mins', 'value': 4758},
                         'duration_in_traffic': {'text': '1 hour 28 mins',
                                                 'value': 5269},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Santa Clara, CA, USA'],
 'origin_addresses': ['Marin County, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '135 km', 'value': 135400},
                         'duration': {'text': '1 hour 40 mins', 'value': 6021},
                         'duration_in_traffic': {'text': '2 hours 37 mins',
                                                 'value': 9435},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['San Mateo, CA, USA'],
 'origin_addresses': ['Marin County

{'destination_addresses': ['Santa Clara, CA, USA'],
 'origin_addresses': ['San Francisco, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '87.4 km', 'value': 87417},
                         'duration': {'text': '57 mins', 'value': 3411},
                         'duration_in_traffic': {'text': '1 hour 25 mins',
                                                 'value': 5072},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['San Mateo, CA, USA'],
 'origin_addresses': ['San Francisco, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '31.1 km', 'value': 31052},
                         'duration': {'text': '27 mins', 'value': 1614},
                         'duration_in_traffic': {'text': '47 mins',
                                                 'value': 2831},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['San Francisco, CA, USA'],
 'origin_addresses': ['San Francisco, CA, USA'],
 'rows

{'destination_addresses': ['San Mateo, CA, USA'],
 'origin_addresses': ['Santa Clara, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '43.9 km', 'value': 43921},
                         'duration': {'text': '33 mins', 'value': 1974},
                         'duration_in_traffic': {'text': '59 mins',
                                                 'value': 3549},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['San Francisco, CA, USA'],
 'origin_addresses': ['Santa Clara, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '72.8 km', 'value': 72828},
                         'duration': {'text': '53 mins', 'value': 3170},
                         'duration_in_traffic': {'text': '1 hour 45 mins',
                                                 'value': 6291},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Napa County, CA, USA'],
 'origin_addresses': ['Santa Clara, CA, USA'],
 'rows': [{'

{'destination_addresses': ['San Francisco, CA, USA'],
 'origin_addresses': ['Sonoma, CA 95476, USA'],
 'rows': [{'elements': [{'distance': {'text': '71.5 km', 'value': 71467},
                         'duration': {'text': '1 hour 4 mins', 'value': 3854},
                         'duration_in_traffic': {'text': '1 hour 47 mins',
                                                 'value': 6434},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Napa County, CA, USA'],
 'origin_addresses': ['Sonoma, CA 95476, USA'],
 'rows': [{'elements': [{'distance': {'text': '56.3 km', 'value': 56254},
                         'duration': {'text': '56 mins', 'value': 3381},
                         'duration_in_traffic': {'text': '1 hour 3 mins',
                                                 'value': 3802},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Marin County, CA, USA'],
 'origin_addresses': ['Sonoma, CA 95476, U

In [28]:
#converting units into minutes/miles
#minute_traffic = (traffic_duration-trip_duration)/60
traffic_duration = [x/60 for x in traffic_duration]
trip_duration  = [x/60 for x in trip_duration]
minute_traffic = [x-y for  x,y in zip(traffic_duration,trip_duration)]
#miles_distance = trip_distance/1,609.344
miles_distance = [z* 0.00062137 for z in trip_distance]
print( len(origin_Loc))

81


In [29]:
traffic_dic = {"Origin":origin_Loc,
               "Destination":destination_loc,
               "Distance(Miles)":miles_distance,
               "Trip length(minutes)":trip_duration,
               "Trip in traffic(minutes)":traffic_duration,
               "Traffic(minutes)":minute_traffic,"Origin Lat" :origin_lat, "Origin Lng" :origin_long,
               "Destination Lat":dest_lat,"Destination Lng" :dest_long
              }
traffic_df = pd.DataFrame(traffic_dic)
traffic_df

,Origin,Destination,Distance(Miles),Trip length(minutes),Trip in traffic(minutes),Traffic(minutes),Origin Lat,Origin Lng,Destination Lat,Destination Lng
0,"Alameda, CA, USA","Sonoma, CA 95476, USA",52.772333,63.100000,93.516667,30.416667,37.765206,-122.241636,38.291859,-122.458036
1,"Alameda, CA, USA","Solano County, CA, USA",57.146156,66.033333,79.100000,13.066667,37.765206,-122.241636,38.310497,-121.901795
2,"Alameda, CA, USA","Santa Clara, CA, USA",39.895061,46.733333,102.133333,55.400000,37.765206,-122.241636,37.354108,-121.955236
3,"Alameda, CA, USA","San Mateo, CA, USA",27.875901,33.466667,69.533333,36.066667,37.765206,-122.241636,37.562992,-122.325525
4,"Alameda, CA, USA","San Francisco, CA, USA",16.355701,27.450000,58.683333,31.233333,37.765206,-122.241636,37.774929,-122.419415
5,"Alameda, CA, USA","Napa County, CA, USA",65.579390,82.033333,93.066667,11.033333,37.765206,-122.241636,38.502469,-122.265389
6,"Alameda, CA, USA","Marin County, CA, USA",46.464806,63.166667,89.900000,26.733333,37.765206,-122.241636,38.083403,-122.763304
7,"Alameda, CA, USA","Contra Costa County, CA, USA",38.544824,63.533333,83.716667,20.183333,37.765206,-122.241636,37.853409,-121.901795
8,"Alameda, CA, USA","Alameda, CA, USA",0.000000,0.000000,0.233333,0.233333,37.765206,-122.241636,37.765206,-122.241636
9,"Contra Costa County, CA, USA","Sonoma, CA 95476, USA",59.955370,84.400000,117.083333,32.683333,37.853409,-121.901795,38.291859,-122.458036


In [74]:
gmaps.configure(api_key=gkey)
locations = traffic_df[['Origin Lat','Origin Lng']].astype(float)
destinations = traffic_df[['Destination Lat','Destination Lng']].astype(float)
traffic_heat = traffic_df['Traffic(minutes)'].astype(float)
#,'Destination Lat','Destination Lng'
start = (37.9,-121.9)
end = (37.6,-122.5)
start2 = (origin_lat[0],origin_long[0])
end2 = (37.354108,-121.955236)
#fig = gmaps.figure(center=(37.765206,-122.241636),zoom_level=10)
#fig.add_layer(gmaps.directions_layer(start,end))
#fig.add_layer(gmaps.directions_layer(start2,end2))
c= 0
while c < 3:
    for x,y,z,q in zip(origin_lat,origin_long,dest_lat,dest_long):
        fig = gmaps.figure(center=(37.765206,-122.241636),zoom_level=10)
        orig = (x,y)
        dest = (z,q)
        route = gmaps.directions_layer(orig,dest)
        fig.add_layer(route)
        c +=1
        time.sleep(5)
    #heat_layer = gmaps.heatmap_layer(locations, weights= traffic_heat, 
                                 #dissipating=False, max_intensity=100,
                                 #point_radius = 5)
#heat_layer.dissipating = False
#heat_layer.max_intensity = 200
#heat_layer.point_radius = .25
#fig.add_layer(heat_layer)

fig


AssertionError: 

In [66]:
print(gmaps.directions_layer(start2,end2))

Directions(data_bounds=[(37.354108, -122.2416355), (37.7652065, -121.955236)], end=(37.354108, -121.955236), start=(37.7652065, -122.2416355), stroke_color='#0088ff')


In [ ]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E","B25077_001E"),{'for': 'zip code tabulation area:*'})
#state_zipcode
#{'for': 'zip code tabulation area:*'}
census_pd = pd.DataFrame(census_data)
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count","NAME": "Name", "zip code tabulation area": "ZIP",
                                      "B25077_001E":"Median home value"})


census_pd.head()

In [ ]:
city_census =pd.merge(census_pd,zip_code_df,how= 'right',on='ZIP')
new_df =city_census.drop(columns=["Poverty Count","Name","Median Age","the_geom","Unemployment Count"],axis=1)
new_df.head()